<a href="https://colab.research.google.com/github/faridlyk/music-genre-classifier-cnn-dnn/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerias

In [ ]:
!pip install tensorflow

In [ ]:
!pip install yt-dlp
!pip install pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 27.1 MB/s eta 0:00:00


In [ ]:
import librosa
import os
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from moviepy.editor import AudioFileClip
import matplotlib.pyplot as plt
import yt_dlp
from pydub import AudioSegment
import joblib

  if event.key is 'enter':



# Código no usado

In [ ]:
# Extract audio features

"""
Extraction of audio features with the librosa library.
NOT USED IN THE PROJECT.
"""

def extract_features(file_path):
  try:
    audio, sample_rate = librosa.load(file_path, duration=30)

    features = {
        'mfccs': np.mean(librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40), axis=0),
        'spectral_centroids': np.mean(librosa.feature.spectral_centroid(y=audio, sr=sample_rate)),
        'chroma_stft': np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate)),
        'melspectrogram': np.mean(librosa.feature.melspectrogram(y=audio, sr=sample_rate)),
        'contrast': np.mean(librosa.feature.spectral_contrast(y=audio, sr=sample_rate)),
        'rolloff': np.mean(librosa.feature.spectral_rolloff(y=audio, sr=sample_rate)),
        'zcr': np.mean(librosa.feature.zero_crossing_rate(y=audio)),
        'tempo': np.mean(librosa.feature.rhythm.tempo(y=audio, sr=sample_rate))
    }
    mfccs_mean = np.mean(features['mfccs'])

    features_array = np.array([
        mfccs_mean,
        features['spectral_centroids'],
        features['chroma_stft'],
        features['melspectrogram'],
        features['contrast'],
        features['rolloff'],
        features['zcr'],
        features['tempo']
    ])

    return features_array

  except Exception as e:
    print(f"Error extracting features from {file_path}: {str(e)}")
    return None

In [ ]:
"""
Processes each file within each folder.
NOT USED IN THE PROJECT.
"""

def process_data(data_path):
    features = []
    labels = []

    # Process each genre folder
    for genre in os.listdir(data_path):
        genre_path = os.path.join(data_path, genre)
        if os.path.isdir(genre_path):
            print(f"Processing {genre} files...")

            # Process each audio file in the genre folder
            for file_name in os.listdir(genre_path):
                if file_name.endswith('.wav'):
                    file_path = os.path.join(genre_path, file_name)
                    extracted_features = extract_features(file_path)

                    if extracted_features is not None and len(extracted_features) > 0:
                        features.append(extracted_features)
                        labels.append(genreToNumber(genre))

    print("Data processing completed.")

    return np.array(features), np.array(labels)

# Funciones

In [ ]:
def genreToNumber(genre):
    genre_dict = {
        'blues': 0,
        'classical': 1,
        'country': 2,
        'disco': 3,
        'hiphop': 4,
        'jazz': 5,
        'metal': 6,
        'pop': 7,
        'reggae': 8,
        'rock': 9
    }
    return genre_dict.get(genre, 9)

In [ ]:
# Extracting the features from the dataset file (59 features)
def process_csv(file_path):
  df = pd.read_csv(file_path)

  # Archivo corrupto
  df = df[df['filename'] != 'jazz.00054.wav']

  # Changes gender names to numbers
  labels = df['label'].apply(genreToNumber)

  # features = df.filter(regex='mean$')
  features = df.filter(regex='(mean$|var$)|(^tempo$)')
  print("Data features processing completed.\n")

  return features, labels

# file_path = '/content/drive/MyDrive/datasets/Data/features_30_sec.csv'
# features, labels = process_csv(file_path)

# if features is not None and labels is not None:
#     print("Features shape:", features.shape)
#     print("Labels shape:", labels.shape)
#     print('\n')
#     print(features)

In [ ]:
# Pre-procesamiento de los espectrogramas de Mel
def extract_img_features(img_path):

  image = Image.open(img_path).convert('L')
  image = image.resize((128, 128))
  image_array = np.array(image)

  # Normalizes data from 0..1
  image_array = image_array / 255.0
  image_array = np.expand_dims(image_array, axis=-1)

  return image_array

In [ ]:
def process_img_data(img_path):
    img_features = []

    # Process each genre folder
    for genre in os.listdir(img_path):
        genre_path = os.path.join(img_path, genre)
        if os.path.isdir(genre_path):
            # print(f"Processing {genre} images files...")

            # Process each image file in the genre folder
            for img_name in os.listdir(genre_path):
                if img_name.endswith('.png'):
                    file_path = os.path.join(genre_path, img_name)
                    extracted_img_features = extract_img_features(file_path)

                    if extracted_img_features is not None and len(extracted_img_features) > 0:
                        img_features.append(extracted_img_features)

    print("Images processing completed.\n")

    return np.array(img_features)



# Código central

In [ ]:
if __name__ == "__main__":
  DATA_PATH = '/content/drive/MyDrive/datasets/Data/genres_original'
  IMG_PATH = '/content/drive/MyDrive/datasets/mel_img'
  FEATURES_PATH = '/content/drive/MyDrive/datasets/Data/features_30_sec.csv'

  data_features, labels = process_csv(FEATURES_PATH)
  img_features = process_img_data(IMG_PATH)

  # print("Features shape:", data_features.shape)
  # print("Labels shape:", labels.shape)
  # print('Images shape:', img_features.shape)

  X_train_features, X_val_features, X_train_images, X_val_images, y_train, y_val = train_test_split(
      data_features, img_features, labels, test_size=0.2, random_state=17
  )

  feature_scaler = StandardScaler()
  X_train_features = feature_scaler.fit_transform(X_train_features)
  X_val_features = feature_scaler.transform(X_val_features)
  joblib.dump(feature_scaler, '/content/drive/MyDrive/datasets/scalers/feature_scaler.pkl')

  input_features = layers.Input(shape=(57,))
  input_images = layers.Input(shape=(128, 128, 1))

  # Dense layer for features
  x1 = layers.Dense(64, activation='relu')(input_features)
  x1 = layers.Dense(32, activation='relu')(x1)

  # Convolutional layer for images
  x2 = layers.Conv2D(32, (3, 3), activation='relu')(input_images)
  x2 = layers.MaxPooling2D((2, 2))(x2)
  x2 = layers.Conv2D(64, (3, 3), activation='relu')(x2)
  x2 = layers.MaxPooling2D((2, 2))(x2)
  x2 = layers.Flatten()(x2)

  combined = layers.concatenate([x1, x2])

  # Final dense layer
  x = layers.Dense(64, activation='relu')(combined)
  output = layers.Dense(10, activation='softmax')(x)

  # Create the model
  model = keras.Model(inputs=[input_features, input_images], outputs=output)
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  # Training the model
  history = model.fit(
      [X_train_features, X_train_images],
      y_train,
      validation_data=([X_val_features, X_val_images], y_val),
      epochs=60,
      batch_size=32
  )

  # Model evaluation
  loss, accuracy = model.evaluate([X_val_features, X_val_images], y_val)
  print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

  model.save('/content/drive/MyDrive/datasets/models/modelv3.h5')

Data features processing completed.

Images processing completed.

Epoch 1/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 26s 964ms/step - accuracy: 0.1208 - loss: 2.4501 - val_accuracy: 0.2300 - val_loss: 2.0993
Epoch 2/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 37s 775ms/step - accuracy: 0.3557 - loss: 1.9683 - val_accuracy: 0.4300 - val_loss: 1.6610
Epoch 3/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 19s 782ms/step - accuracy: 0.5062 - loss: 1.4266 - val_accuracy: 0.5000 - val_loss: 1.2776
Epoch 4/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 20s 774ms/step - accuracy: 0.5869 - loss: 1.1483 - val_accuracy: 0.5600 - val_loss: 1.1694
Epoch 5/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 20s 812ms/step - accuracy: 0.7183 - loss: 0.8743 - val_accuracy: 0.6550 - val_loss: 1.0334
Epoch 6/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 22s 867ms/step - accuracy: 0.7240 - loss: 0.7417 - val_accuracy: 0.6000 - val_loss: 1.0942
Epoch 7/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 823ms/step - accuracy: 0.7912 - loss: 0.6119 - val_accuracy: 0.6050 - val_loss: 1.0811
Epoch 8/60
25/25 ━━━━━━━━━━━━━

Loss: 0.9979, Accuracy: 0.7750


# Cargar modelos

In [ ]:
# Loading the first model
model = tf.keras.models.load_model('/content/drive/MyDrive/datasets/models/model.h5')

DATA_PATH = '/content/drive/MyDrive/datasets/Data/genres_original'
IMG_PATH = '/content/drive/MyDrive/datasets/Data/images_original'
FEATURES_PATH = '/content/drive/MyDrive/datasets/Data/features_30_sec.csv'

data_features, labels = process_csv(FEATURES_PATH)
img_features = process_img_data(IMG_PATH)

features_standardized = StandardScaler().fit_transform(data_features)

X_train_features, X_val_features, X_train_images, X_val_images, y_train, y_val = train_test_split(
    features_standardized, img_features, labels, test_size=0.2, random_state=17
)

loss, accuracy = model.evaluate([X_val_features, X_val_images], y_val)
print(f"\nAccuracy: {accuracy * 100:.2f}%")

Data features processing completed.

Images processing completed.

7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 185ms/step - accuracy: 0.6873 - loss: 1.6605

Accuracy: 68.50%


In [ ]:
# Loading the second model
model = tf.keras.models.load_model('/content/drive/MyDrive/datasets/models/modelv2.h5')

DATA_PATH = '/content/drive/MyDrive/datasets/Data/genres_original'
IMG_PATH = '/content/drive/MyDrive/datasets/mel_img'
FEATURES_PATH = '/content/drive/MyDrive/datasets/Data/features_30_sec.csv'

data_features, labels = process_csv(FEATURES_PATH)
img_features = process_img_data(IMG_PATH)

features_standardized = StandardScaler().fit_transform(data_features)

X_train_features, X_val_features, X_train_images, X_val_images, y_train, y_val = train_test_split(
    features_standardized, img_features, labels, test_size=0.2, random_state=17
)

loss, accuracy = model.evaluate([X_val_features, X_val_images], y_val)
print(f"\nAccuracy: {accuracy * 100:.2f}%")

Data features processing completed.

Images processing completed.

7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 192ms/step - accuracy: 0.7876 - loss: 1.0404

Accuracy: 79.50%


# Código extra

In [ ]:
def extract_audio_features(audio_path):
  y, sr = librosa.load(audio_path)

  features = {}

  features['chroma_stft_mean'] = np.mean(librosa.feature.chroma_stft(y=y, sr=sr))
  features['chroma_stft_var'] = np.var(librosa.feature.chroma_stft(y=y, sr=sr))
  features['rms_mean'] = np.mean(librosa.feature.rms(y=y))
  features['rms_var'] = np.var(librosa.feature.rms(y=y))
  features['spectral_centroid_mean'] = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
  features['spectral_centroid_var'] = np.var(librosa.feature.spectral_centroid(y=y, sr=sr))
  features['spectral_bandwidth_mean'] = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
  features['spectral_bandwidth_var'] = np.var(librosa.feature.spectral_bandwidth(y=y, sr=sr))
  features['rolloff_mean'] = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
  features['rolloff_var'] = np.var(librosa.feature.spectral_rolloff(y=y, sr=sr))
  features['zero_crossing_rate_mean'] = np.mean(librosa.feature.zero_crossing_rate(y=y))
  features['zero_crossing_rate_var'] = np.var(librosa.feature.zero_crossing_rate(y=y))
  features['harmony_mean'] = np.mean(librosa.effects.harmonic(y))
  features['harmony_var'] = np.var(librosa.effects.harmonic(y))
  features['perceptr_mean'] = np.mean(librosa.effects.percussive(y))
  features['perceptr_var'] = np.var(librosa.effects.percussive(y))

  tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
  features['tempo'] = tempo

  mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
  for i in range(1, 21):
    features[f'mfcc{i}_mean'] = np.mean(mfccs[i-1])
    features[f'mfcc{i}_var'] = np.var(mfccs[i-1])

  return pd.DataFrame(features)

In [ ]:
def download_audio(link):
  OUTPUT_DIR = '/content/sample_data'

  ydl_opts = {
      'format': 'bestaudio/best',
      'outtmpl': os.path.join(OUTPUT_DIR, '%(title)s.%(ext)s'),
      'quiet': True
  }

  try:
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
      info_dict = ydl.extract_info(link, download=True)
      title = info_dict.get('title', None)

      downloaded_file = os.path.join(OUTPUT_DIR, str(title) + '.webm')
      if not os.path.exists(downloaded_file):
        print(f"Error: Downloaded file not found at {downloaded_file}")
        return None

      audio = AudioSegment.from_file(os.path.join(OUTPUT_DIR, str(title) + '.webm'), format='webm')
      audio.export(os.path.join(OUTPUT_DIR, str(title) + '.wav'), format='wav')
      os.remove(os.path.join(OUTPUT_DIR, str(title) + '.webm'))
      print('Audio downloaded')

    return os.path.join(OUTPUT_DIR, str(title) + '.wav'), title

  except Exception as e:
    print(f"There was a problem downloading the video: {e}")
    return None

In [ ]:
def extract_clip(input_file, output_file):
  audio = AudioFileClip(input_file)

  audio_duration = audio.duration
  start_time = (audio_duration - 30) / 2
  end_time = start_time + 30

  clip = audio.subclip(start_time, end_time)
  clip.write_audiofile(output_file, verbose=False, logger=None)

  os.remove(input_file)
  print(f"Clip extracted successfully\n")

  return output_file

In [ ]:
def extract_mel_img(clip):
  y, sr = librosa.load(clip)
  mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
  mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)

  img = (mel_spectrogram_db - mel_spectrogram_db.min()) / (mel_spectrogram_db.max() - mel_spectrogram_db.min())

  img_grayscale = Image.fromarray((img * 255).astype(np.uint8)).convert('L')
  img_colored = np.stack((img,) * 3, axis=-1)
  img_colored = (img_colored * 255).astype(np.uint8)
  image = Image.fromarray(img_colored, mode='RGB')

  image.save(clip.replace('.wav', '.png'))

  return clip.replace('.wav', '.png')

In [ ]:
def predict_genre(clip_path, model):
  audio_features = extract_audio_features(clip_path)

  img_path = extract_mel_img(clip_path)
  img_features = extract_img_features(img_path)

  feature_scaler = joblib.load('/content/drive/MyDrive/datasets/scalers/feature_scaler.pkl')
  features_standardized = feature_scaler.transform(audio_features)

  prediction = model.predict([features_standardized, np.expand_dims(img_features, axis=0)])

  genre_dict = {
      0: 'blues',
      1: 'classical',
      2: 'country',
      3: 'disco',
      4: 'hiphop',
      5: 'jazz',
      6: 'metal',
      7: 'pop',
      8: 'reggae',
      9: 'rock'
  }

  top_3_indices = np.argpartition(prediction[0], -3)[-3:]
  top_3_indices = top_3_indices[np.argsort(-prediction[0][top_3_indices])]

  genres = [genre_dict[i] for i in top_3_indices]
  probs = prediction[0][top_3_indices]

  predicted_genre_index = np.argmax(prediction)
  predicted_genre = genre_dict[predicted_genre_index]

  return predicted_genre, list(zip(genres, probs))

In [ ]:
PATH = '/content/sample_data'

model = tf.keras.models.load_model('/content/drive/MyDrive/datasets/models/modelv2.h5')

URL = input('\nIngresa la URL de YouTube: ').strip()
audio_path, title = download_audio(URL)

audio_clip_path = extract_clip(audio_path, os.path.join(PATH, 'clip.wav'))

predicted_genre, predictions = predict_genre(audio_clip_path, model)

os.remove(audio_clip_path)
os.remove(audio_clip_path.replace('.wav', '.png'))

print(f'\nVideo: {title}')
print(f"Género predicho: {predicted_genre}")

print("\nTop 3 predicted genres:")
for genre, prob in predictions:
    print(f"{genre}: {prob:.2%}")